# **附录1：灯光配置**

## **1 WS281x 灯带：使用树莓派直接驱动**

#### **依赖：**

+ [rpi_ws281x](https://github.com/jgarff/rpi_ws281x)

    [python-binding](https://github.com/rpi-ws281x/rpi-ws281x-python)


#### **灯带与树莓派的连接：**

+ [树莓派连接 NeoPixel 指南 (Adafruit)](https://cdn-learn.adafruit.com/downloads/pdf/neopixels-on-raspberry-pi.pdf)

+ [Raspberry Pi &
WS2812 LED-Strips](https://dordnung.de/raspberrypi-ledstrip/ws2812)


#### **root 权限运行：**

因为控制灯光需要 sudo 权限，所以安装 python 依赖时都要以 sudo 权限安装；

jupyter-lab 的启动也要以 root 权限启动：`sudo -E env "PATH=$PATH" jupyter lab --allow-root`，参考[这里](https://stackoverflow.com/questions/41622988/root-access-for-jupyter-ipython-notebook)


####  **更多的灯光效果可以参考[这里定义的函数](https://github.com/BSFEMA/RPi_WS2812/blob/master/function_library.py)**

&emsp;

In [ ]:
# 导入依赖库，输入参数值
from rpi_ws281x import PixelStrip, Color

LED_COUNT = 64              # 接入灯带灯珠的数量
LED_PIN = 18                # 数据写入线连接树莓派 GPIO 18
LED_FREQ_HZ = 800000        # 默认参数，参照依赖库中的示例代码直接使用
LED_DMA = 10                # 默认参数，参照依赖库中的示例代码直接使用
LED_BRIGHTNESS = 200        # 灯珠的亮度，范围0-255
LED_INVERT = False          # 默认参数，参照依赖库中的示例代码直接使用
LED_CHANNEL = 0             # 默认参数，参照依赖库中的示例代码直接使用



# 按以上参数初始化
strip = PixelStrip(LED_COUNT, LED_PIN, LED_FREQ_HZ, LED_DMA, LED_INVERT, LED_BRIGHTNESS, LED_CHANNEL)
strip.begin()



# 以函数的方式简单定义一种灯光效果，就是让所有灯珠呈现某一种颜色
# 颜色直接使用依赖库现成的函数 Color(r,g,b) 定义
def ColorWipe(strip, color):
    for i in range(strip.numPixels()):
        strip.setPixelColor(i, color)
    strip.show()

    
# 让灯带呈现红色
ColorWipe(strip, Color(255,0,0))

In [ ]:
import numpy as np
import time
import random 
from rpi_ws281x import PixelStrip, Color
from madmom.features.beats import DBNBeatTrackingProcessor, RNNBeatProcessor
from madmom.models import BEATS_LSTM
from madmom.processors import IOProcessor, process_online
from codelab_adapter_client import AdapterNode



# 在 EIM node 中定义接收和发送消息的函数
class MyNode(AdapterNode):
    NODE_ID = "eim/tracktheBeat"     # 这个 ID 名（即 "eim/"后面的部分自己定义，配套 Scratch 项目中使用的接收和发送 EIM 消息的积木内要写与此一致的名字
    
    def __init__(self):
        super().__init__()
        self.is_ready = False
        
    def send_data(self, content):    # 向 Scratch 发送消息用
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)
        
    def extension_message_handle(self, topic, payload):      # 接收 Scratch 发过来的消息
        self.logger.info(f'the message payload from scratch: {payload}')
        content = payload["content"]
        if content == "ready":                  # content 内容自定义，就是 Scratch 发送过来的消息，这里是字符 “ready”
            self.is_ready = True
        
node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)

# 当 python 代码被启动运行后会卡在这里等待来自 Scratch 的消息
# 接收到 Scratch 发来的 ready 后就会向下继续
while not node.is_ready:         
    print("等待开始")
    time.sleep(0.5)

print("go")



# 下面是基于 rpi_wx281x 库设置的灯带基本参数，参考库内用例
LED_COUNT = 30              # 灯珠数量
LED_PIN = 18                # 灯带写入数据线连接的树莓派 pin：GPIO 18
LED_FREQ_HZ = 800000        # 默认参数
LED_DMA = 10                # 默认参数
LED_BRIGHTNESS = 200        # 灯珠的亮度，范围 0-255，自己根据需要修改
LED_INVERT = False          # 默认参数
LED_CHANNEL = 0             # 默认参数


# 根据以上参数初始化灯带
strip = PixelStrip(LED_COUNT, LED_PIN, LED_FREQ_HZ, LED_DMA, LED_INVERT, LED_BRIGHTNESS, LED_CHANNEL)
strip.begin()


# 利用 madmom 实时分析音乐节拍所需的参数
kwargs = dict(
    fps = 100,
    correct = True,
    infile = None,
    outfile = None,
    #max_bpm = 170,
    #min_bpm = 100,
    nn_files = [BEATS_LSTM[0],BEATS_LSTM[1], BEATS_LSTM[2]],
    num_frames = 1,
    online = True,
)



# 对应 Scratch 项目，预定义 8 种颜色
color_list = ["red", "orange", "yellow", "green", "cyan", "blue", "purple", "violet"]



# 灯带颜色效果函数定义，很简单，就是让所有灯珠呈现某种颜色
def ColorWipe(strip, color):
    for i in range(strip.numPixels()):
        strip.setPixelColor(i, color)
    strip.show()  
    
    
    
# 每当节拍被识别到后，就会调用下面这个函数
# 会随机从 8 种颜色中选取一种，使灯光渲染颜色，Scratch 中对应颜色按钮闪动
def beat_callback(beats, output=None):
    if len(beats) > 0:
        color = random.choice(color_list)
        print(color)
        if color == 'red':
            node.send_data("red")
            ColorWipe(strip, Color(255, 0, 0))
        if color == 'blue':
            node.send_data("blue")
            ColorWipe(strip, Color(0, 0, 255))
        if color == 'green':
            node.send_data("green")
            ColorWipe(strip, Color(0, 255, 0))
            
        if color == 'purple':
            node.send_data("purple")
            ColorWipe(strip, Color(255, 0, 255))
            
        if color == 'yellow':
            node.send_data("yellow")
            ColorWipe(strip, Color(255, 255, 0))
            
        if color == 'cyan':
            node.send_data("cyan")
            ColorWipe(strip, Color(0, 180, 100))
            
        if color == 'violet':
            node.send_data("violet")
            ColorWipe(strip, Color(187, 51, 133))
            
        if color == 'orange':
            node.send_data("orange")
            ColorWipe(strip, Color(255, 85, 0))
            
 
  
# madmom 实时分析音乐节拍用到的 processor            
    
in_processor = RNNBeatProcessor(**kwargs)
beat_processor = DBNBeatTrackingProcessor(**kwargs)
out_processor = [beat_processor, beat_callback]
processor = IOProcessor(in_processor, out_processor)
process_online(processor, **kwargs)